In [1]:
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path,pdfinfo_from_path
import os
import time
import nltk.data

Define the input and output file name

In [7]:
start_time = time.time()
# Path of the pdf
PDF_file = "books/Harry_potter_1.pdf"
# Creating a text file to write the output
outfile = "output_"+str(PDF_file.split(".")[0])+".txt"

Converting PDF to images

In [3]:
image_counter = 1
# Counter to store images of each page of PDF to image
info = pdfinfo_from_path(PDF_file, userpw=None, poppler_path=None)

maxPages = 10
page_interval=10
for page in range(1, maxPages+1, page_interval) : 
    # Store all the pages of the PDF in a variable
    pages=convert_from_path(PDF_file, dpi=700, first_page=page, last_page = min(page+page_interval-1,maxPages))
  
    # Iterate through all the pages stored above
    for page in pages:
      
        #image file name
        filename = str(PDF_file.split(".")[0])+"_page_"+str(image_counter)+".jpg"
          
        # Save the image of the page in system
        page.save(filename, 'JPEG')

        # Increment the counter to update filename
        image_counter = image_counter + 1
print("--- %s seconds ---" % (time.time() - start_time))

--- 15.862645864486694 seconds ---


Recognizing text from the images using OCR

In [11]:
# Variable to get count of total number of pages
filelimit = image_counter-1

# Open the file in append mode so that 
# All contents of all images are added to the same file
f = open(outfile, "w")
  
# Iterate from 1 to total number of pages
for i in range(1, filelimit + 1):
  

    #filename = "page_"+str(i)+".jpg"
    filename = str(PDF_file.split(".")[0])+"_page_"+str(i)+".jpg"
          
    # Recognize the text as string in image using pytesserct
    text = str(((pytesseract.image_to_string(Image.open(filename),config=r'--tessdata-dir "C:\Program Files\Tesseract-OCR\tessdata"'))))
  
   
    text = text.replace('-\n', '') 
    text = text.replace('', '') 
  
    # Finally, write the processed text to the file.
    f.write(text)
    
    #deleting the image files already processed
    os.remove(filename) 
  
# Close the file after writing all the text.
f.close()
print("--- %s seconds ---" % (time.time() - start_time))

--- 192.05776000022888 seconds ---


Get output as a sentence in a line

In [ ]:
a_file = open(outfile, "r",encoding="ISO-8859-1")
lines = a_file.readlines()
a_file.close()


new_file = open(outfile, "w",encoding="ISO-8859-1")
for line in lines:
    if not (line.startswith('Page |')):
        line=line.replace('\n',' ')
        new_file.write(line)
new_file.close()

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
fp = open(outfile,"r",encoding="ISO-8859-1")
data = fp.read()
fp.close()
text='\n'.join(tokenizer.tokenize(data))
f=open(outfile, "w",encoding="ISO-8859-1")
#print(text)
f.write(text)
f.close()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import pandas as pd

data = pd.read_fwf(outfile, header = None, delimiter='\n')
print(data)